In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 14:19:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price_4_bed, YEAR(date) as year
    from home_sales
    WHERE bedrooms = 4 
    GROUP BY YEAR(date)
    """).show()


+---------------+----+
|avg_price_4_bed|year|
+---------------+----+
|      296363.88|2022|
|       300263.7|2019|
|      298353.78|2020|
|      301819.44|2021|
+---------------+----+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price_3_bed, date_built
    from home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 
    GROUP BY date_built
    """).show()

+---------------+----------+
|avg_price_3_bed|date_built|
+---------------+----------+
|      290555.07|      2016|
|      293683.19|      2012|
|      292676.79|      2017|
|      290852.27|      2014|
|      295962.27|      2013|
|      291117.47|      2011|
|       288770.3|      2015|
|      292859.62|      2010|
+---------------+----------+



In [9]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price_2_floor, date_built
    from home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living>= 2000
    GROUP BY date_built
    """).show()

+-----------------+----------+
|avg_price_2_floor|date_built|
+-----------------+----------+
|         293965.1|      2016|
|        307539.97|      2012|
|        280317.58|      2017|
|        298264.72|      2014|
|        303676.79|      2013|
|        276553.81|      2011|
|        297609.97|      2015|
|        285010.22|      2010|
+-----------------+----------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT view, 
    ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price)>= 350000
    """).show()  

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 2.639008045196533 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT view, 
    ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price)>= 350000
    """).show()  


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 0.862846851348877 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partitioned")

In [15]:
# 11. Read the formatted parquet data.
parq_df = spark.read.parquet("date_built_partitioned")

In [16]:
# 12. Create a temporary table for the parquet data.
parq_df.createOrReplaceTempView("parquet_data")

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("""
    SELECT view, 
    ROUND(AVG(price), 2) AS avg_price
    FROM parquet_data
    GROUP BY view
    HAVING AVG(price)>= 350000
    """).show()  

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  73| 752861.18|
|  87| 1072285.2|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 1.9589171409606934 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False